In [2]:
import numpy as np
import random
from collections import deque


In [3]:
class State:
    num_cells = 16
    size = 4

    first_row_solved = None
    second_row_solved = None
    full_solved = None

    def __init__(self,numbers):
        self.numbers = np.array(numbers,dtype=int)
        self.solved = self.count_solved_rows()
        self.value = 0.0
        self.policy = None
        self.terminal = self.solved == 4
        self.empty_index = np.where(self.numbers == 16)[0][0]

    def count_solved_rows(self):
        for i in range(16):
            if self.numbers[i] != i+1:
                if i<4:
                    return 0
                elif i<8:
                    return 1
                return 2
        return 4
    
    def possible_actions(self):
        actions = []
        r = self.empty_index//4
        c = self.empty_index%4
        if r+1 < 4:
            actions.append(self.empty_index+4)
        if r-1 >= 0:
            actions.append(self.empty_index-4)
        if c+1 < 4:
            actions.append(self.empty_index+1)
        if c-1 >= 0:
            actions.append(self.empty_index-1)
        return actions
    
    def next_state(self,action):
        new_numbers = self.numbers.copy()
        new_numbers[self.empty_index] = new_numbers[action]
        new_numbers[action] = 16
        return State(new_numbers)
    

In [4]:
State.first_row_solved = State([1,2,3,4,16,0,0,0,0,0,0,0,0,0,0,0])
State.second_row_solved = State([1,2,3,4,5,6,7,8,16,0,0,0,0,0,0,0])
State.full_solved = State([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16])

In [5]:
class PuzzleSolver:
    def __init__(self,γ = 0.9,theta = 0.1):
        self.gamma = γ
        self.theta = theta
        self.states = {}
        self.policy = {}
    
    def generate_states_from(self,state): #generating all the states from a given state
        queue = deque([state])
        while queue:
            current = queue.popleft()
            if tuple(current.numbers) in self.states:
                continue
            self.states[tuple(current.numbers)] = current
            for a in current.possible_actions():
                new_state = current.next_state(a)
                queue.append(new_state)

    def value_iteration(self):
        while True:
            δ = 0
            for s in self.states.values():
                if s.terminal:
                    continue
                v = s.value
                action_values = [self.reward(s,s.next_state(a)) + self.gamma * self.states[tuple(s.next_state(a).numbers)].value for a in s.possible_actions()]
                s.value = max(action_values)
                best_action = s.possible_actions()[np.argmax(action_values)]
                self.policy[tuple(s.numbers)] = best_action
                δ = max(δ,abs(v-s.value))
            if δ < self.theta:
                break
            
    def reward(self, current, next):
        if next.terminal:
            return 100
        return next.solved - current.solved
    
    def train(self): #training the agent for optimal actions
        self.generate_states_from(State.first_row_solved)
        self.generate_states_from(State.second_row_solved)
        self.generate_states_from(State.full_solved)
        self.value_iteration()

    def random_state(self):
        s = State.full_solved
        for i in range(100):
            s = s.next_state(random.random.choice(s.possible_actions()))
        return s
    
    def start(self):
        s = self.random_state()
        T = 0
        while not s.terminal:
            print(s.numbers.reshape(4,4))
            a = self.policy[tuple(s.numbers)]
            s = s.next_state(a)
            T += 1
            print('-----------------')
            print(f'T = {T}')
        print(f'Done in {T} steps')



In [ ]:
puzzle = PuzzleSolver()
puzzle.train()
puzzle.start()
